In [1]:
#ruff: noqa
import jax
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.units as u

from jorbit import Particle
from jorbit.accelerations.static_helpers import precompute_perturber_positions
from jorbit.integrators import ias15_evolve, ias15_static_evolve, initialize_ias15_integrator_state, ias15_static_step
from jorbit.accelerations import create_static_default_acceleration_func

In [2]:
p = Particle.from_horizons("274301", Time("2026-01-01"))
state = p.keplerian_state.to_system()

x, v = p.integrate(Time("2026-02-01"))

In [3]:
planet_pos, planet_vel, asteroid_pos, asteroid_vel, all_times, obs_indices, gms = precompute_perturber_positions(
    t0=Time("2026-01-01"),
    times=Time("2026-02-01"),
    max_step_size=1,
    de_ephemeris_version="440"
)

perturber_pos = jnp.concatenate((planet_pos, asteroid_pos), axis=2)
perturber_vel = jnp.concatenate((planet_vel, asteroid_vel), axis=2)

state.fixed_perturber_positions = perturber_pos[0,0]
state.fixed_perturber_velocities = perturber_vel[0,0]
state.fixed_perturber_log_gms = gms

In [4]:
acc_func = create_static_default_acceleration_func()
a0 = acc_func(state)
integrator_init = initialize_ias15_integrator_state(a0)
integrator_init.dt = 1.0

In [5]:
positions, velocities, _, _ = ias15_static_evolve(
    state,
    acc_func,
    jnp.diff(all_times),
    perturber_pos,
    perturber_vel,
    gms,
    integrator_init,
)

In [7]:
positions[-1] - x

Array([[-0.00861124,  0.00626885,  0.00182251]], dtype=float64)

In [8]:
integrator_init.dt

1.0

In [11]:
planet_pos, planet_vel, asteroid_pos, asteroid_vel, all_times, obs_indices, gms = precompute_perturber_positions(
    t0=Time("2026-01-01"),
    times=Time("2026-01-02"),
    max_step_size=5,
    de_ephemeris_version="440"
)

perturber_pos = jnp.concatenate((planet_pos, asteroid_pos), axis=2)
perturber_vel = jnp.concatenate((planet_vel, asteroid_vel), axis=2)

In [12]:
p = Particle.from_horizons("274301", Time("2026-01-01"))
state = p.keplerian_state.to_system()

x, v = p.integrate(Time("2026-01-02"))

In [13]:
state.fixed_perturber_positions = perturber_pos[0,0]
state.fixed_perturber_velocities = perturber_vel[0,0]
state.fixed_perturber_log_gms = gms

acc_func = create_static_default_acceleration_func()
a0 = acc_func(state)
integrator_init = initialize_ias15_integrator_state(a0)
integrator_init.dt = 1

In [ ]:
new_state, new_integrator_init = ias15_static_step(
    state,
    acc_func,
    integrator_init,
    fixed_perturber_positions=perturber_pos[0],
    fixed_perturber_velocities=perturber_vel[0],
    fixed_perturber_log_gms=gms,
)

In [ ]:
new_state.tracer_positions - x, new_state.tracer_velocities - v

(Array([[-2.66453526e-15,  2.44249065e-15,  6.10622664e-16]], dtype=float64),
 Array([[-1.47451495e-17, -6.07153217e-18, -4.77048956e-18]], dtype=float64))

In [ ]:
new_state, new_integrator_init = ias15_static_step(
    new_state,
    acc_func,
    integrator_init,
    fixed_perturber_positions=perturber_pos[3],
    fixed_perturber_velocities=perturber_vel[3],
    fixed_perturber_log_gms=gms,
)

In [ ]:
new_state.tracer_positions

Array([[-1.92845846, -1.41130098, -0.4380735 ]], dtype=float64)

In [ ]:
positions[obs_indices]

Array([[[-1.95958228, -1.38178686, -0.42942657]],

       [[        nan,         nan,         nan]]], dtype=float64)

In [ ]:
from jorbit import Ephemeris

eph = Ephemeris()
dir(eph)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_combined_log_gms',
 '_combined_names',
 'earliest_time',
 'ephs',
 'latest_time',
 'processor',
 'state']